# Optimization notebook
This notebook is to run a multi-objective optimization with the Zambezi model. Two alternatives will be given to that end. First implementation is with the Platypus library which has a fairly simple interface. The second implementation is the EMA Workbench which additionally offers parallelization functionality

In [4]:
import pandas as pd
import numpy as np
import os
os.chdir('../src')
from model_zambezi_OPT import model_zambezi

from platypus import NSGAII, Problem, Real

In [5]:
ZambeziProblem = model_zambezi()

## Platypus (NSGAII)

In [3]:
problem = Problem(ZambeziProblem.Nvar, ZambeziProblem.Nobj)
problem.types[:] = Real(0, 1)
problem.function = ZambeziProblem.evaluate

algorithm = NSGAII(problem=problem, population_size=20)
algorithm.run(100)

c:\Users\dakoluk\AppData\Local\Programs\Python\Python310\lib\site-packages\platypus\algorithms.py:1048: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if indicator is "hypervolume":
c:\Users\dakoluk\AppData\Local\Programs\Python\Python310\lib\site-packages\platypus\algorithms.py:1048: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if indicator is "hypervolume":
c:\Users\dakoluk\AppData\Local\Programs\Python\Python310\lib\site-packages\platypus\algorithms.py:1048: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if indicator is "hypervolume":
c:\Users\dakoluk\AppData\Local\Programs\Python\Python310\lib\site-packages\platypus\algorithms.py:1048: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if indicator is "hypervolume":


KeyboardInterrupt: 

In [6]:
objectives_outcome = dict()
for i, column_name in enumerate(['Hydropower','Environment','Irrigation']):
    objectives_outcome[column_name] = [s.objectives[i] for s in algorithm.result]

objectives_df = pd.DataFrame(objectives_outcome)


In [7]:
print(objectives_df)
from src.various_plots import parallel_plots
parallel_plots(objectives_df)

    Hydropower   Environment  Irrigation
0    15.178700  3.289746e+06    3.594867
1    14.921654  3.733126e+06    1.962184
2    17.575309  2.423616e+06    1.636896
3    16.183941  2.299253e+06    3.502703
4    15.426513  3.098156e+06    2.916923
5    16.699791  2.380324e+06    2.327759
6    16.970426  2.433317e+06    1.836414
7    16.217340  2.307791e+06    2.769851
8    15.235513  3.263515e+06    3.348402
9    15.243236  3.368377e+06    1.761276
10   17.029598  2.420962e+06    1.872167
11   16.832802  2.317158e+06    2.137552
12   16.869286  2.326919e+06    1.910599
13   16.972212  2.428629e+06    1.689669
14   16.964694  2.423462e+06    1.870241
15   16.852162  2.309749e+06    1.940196
16   16.950459  2.383191e+06    2.380581
17   16.352100  2.398968e+06    3.684423
18   15.246359  3.372396e+06    1.950270
19   16.993899  2.427232e+06    1.896631


ModuleNotFoundError: No module named 'src'

## EMA Workbench

In [8]:
def model_wrapper(**kwargs):
    input = [kwargs['v' + str(i)] for i in range(len(kwargs))]
    Hydropower, Environment, Irrigation = tuple(ZambeziProblem.evaluate(np.array(input)))
    return Hydropower, Environment, Irrigation


In [9]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model)

model = Model('zambeziproblem', function=model_wrapper)

model.levers = [RealParameter('v' + str(i), -1, 1) for i in range(ZambeziProblem.Nvar)]

#specify outcomes
model.outcomes = [ScalarOutcome('Hydropower', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Environment', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Irrigation', ScalarOutcome.MINIMIZE)]

## First trying with SequentialEvaluator

In [10]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1, 0.1, 0.1])

  0%|                                                  | 0/100 [00:00<?, ?it/s]c:\Users\dakoluk\Documents\16.02.22.Multiobjective\Multiobjective-multi-reservoir-control-d50e4da0f6a9a9c852b4904e640299adc96714bb\ZambeziSmashPython\src\alternative_policy_structures.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  y = min(input_inflow,input_w*(pow(input_inflow/hdg_dn,m)))
c:\Users\dakoluk\Documents\16.02.22.Multiobjective\Multiobjective-multi-reservoir-control-d50e4da0f6a9a9c852b4904e640299adc96714bb\ZambeziSmashPython\src\alternative_policy_structures.py:33: RuntimeWarning: invalid value encountered in double_scalars
  y = min(input_inflow,input_w*(pow(input_inflow/hdg_dn,m)))
100%|████████████████████████████████████████| 100/100 [45:36<00:00, 27.37s/it]
[MainProcess/INFO] optimization completed, found 13 solutions


## MultiprocessingEvaluator

In [11]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1, 0.1, 0.1])

## Just giving different epsilon values with SequentialEvaluator

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1,]*len(model.outcomes))

## Just giving different epsilon values with MultiprocessingEvaluator

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1,]*len(model.outcomes))

## IpyparallelEvaluator is not gonna work

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, IpyparallelEvaluator, ema_logging
from ema_workbench.em_framework.ema_ipyparallel import (start_logwatcher, set_engine_logger,
                                  initialize_engines, cleanup, _run_experiment)
import dill
from ipyparallel import Client


rc = Client()

ema_logging.log_to_stderr(ema_logging.INFO)

with IpyparallelEvaluator(model, client=rc) as evaluator:
    results = evaluator.optimize(nfe=100, searchover='levers',
    epsilons=[0.1,]*len(model.outcomes))

Waiting for connection file: ~\.ipython\profile_default\security\ipcontroller-client.json


OSError: Connection file '~\\.ipython\\profile_default\\security\\ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.